In [ ]:
import os
import tensorflow as tf
import numpy as np
import utilities as u
import subpixellayer as sl # Is important for the subpixel layer

from keras.models import load_model
from datetime import datetime, timedelta

# Ignore Tensorflow Warnings and other Tensorflow options
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

%matplotlib widget

# Extract time series at given locations

In [ ]:
# Model directory / Network architecture (e.g. DataAugmented, OriginalFukami,...)
model_dir = "Subpixel"

# Additional run info
variables = ["Hs", "Tm02", "Dir"]

grid = (10, 10)
upfactor = 16

# Define the points where the time series are extracted (in pixel)
xcoords = [69, 86, 69]
ycoords = [94, 104, 77] 

# Test data serials
sample_start = 8761
sample_end = 17496
serial = np.arange(sample_start, sample_end+1, 1)

# Training data serials
sample_ref_start = 24
sample_ref_end = 8760
serial_ref = np.arange(sample_ref_start, sample_ref_end+1, 1)

dim = 1

# Save the figure?
save = False

In [ ]:
for point in zip(xcoords, ycoords):
    xi = point[0]
    yi = point[1]
    
    for k, var in enumerate(variables):# Load best model of the run
        fdir = "Models/{}".format(model_dir)
        fmodel = "Model_Inp_{}.hdf5".format(var)
        fmodel = os.path.join(fdir, fmodel)
        model = load_model(fmodel)

        # Prepare file locations for reference and input data
        fname_HR = './Data/HR/{}/BaskCoast_{}_{{}}.npy'
        fname_HR = fname_HR.format(var, var.upper())
        fname_LR = './Data/LR/{}/BaskCoast_{}_{{}}.npy'
        fname_LR = fname_LR.format(var, var.upper())

        # Define output directory
        path_dir = "./Data/Time_Series"
        path_arr = path_dir + "/Var_{}_posxy_{}_{}.npy"
        path_arr = path_arr.format(var, xi, yi)

        # Load the test data set
        X, y = u.load_data((fname_HR, fname_LR), serial, var)
        
        # Convert NaN to zero
        X = np.nan_to_num(X)
        y = np.nan_to_num(y)

        # Evaluate model on whole data set
        prediction = model.predict(X)
        
        # Extract time series at location (adjust position for LR input)
        X = X[:, int(xi/upfactor), int(yi/upfactor), 0]
        y = y[:, xi, yi, 0]
        prediction = prediction[:, xi, yi, 0]
        
        # Shift directional values back if necessary
        X = (X + 255)%360
        y = (y + 255)%360
        prediction = (prediction + 255)%360
        
        # Save the computed data
        if not os.path.isdir(path_dir):
            os.makedirs(path_dir)

        np.save(path_arr, np.r_[prediction, y, X])
        

        # Extract time series for HR training data in similar fashion
        # Prepare file locations for reference and input data
        fname_HR = './Data/HR/{}/BaskCoast_{}_{{}}.npy'
        fname_HR = fname_HR.format(var, var.upper())

        # Define output directory
        path_dir = "./Data/Time_Series"
        path_arr = path_dir + "/Ref_Var_{}_posxy_{}_{}.npy"
        path_arr = path_arr.format(var, xi, yi)

        y = np.zeros((len(serial_ref), grid[0]*upfactor, grid[1]*upfactor, dim))

        # Load reference training data
        for i, s in enumerate(serial_ref):
            y[i,:,:,0] = np.load(fname_HR.format(s))

        # Extract time series at given location
        ref = y[:,xi,yi,0]

        # Save the loaded data
        if not os.path.isdir(path_dir):
            os.makedirs(path_dir)

        np.save(path_arr, ref)